## Project Description
@todo


### Nomenclature
Property names are derived out of entities they join.

The ground/root entity is called ent0
Add the word out/in when required as shown in photo here: 
![title]("resources/nomenclature.png")
@todo


In [1]:
#Importing important stuff
import pickle
import json
import copy

import utils.dbpedia_interface as db_interface
import utils.natural_language_utilities as nlutils

ImportError: No module named utils.dbpedia_interface

In [ ]:
'''
    Initializing some stuff. Namely: DBpedia interface class.
    Reading the list of 'relevant' properties.
'''

In [1]:
'''
	This code is responsible for finding a localsubgraph for every important DBpedia entry (based on pagerank) and using it to fill the templates found in templates.py file	
'''
from pprint import pprint
import pickle
import json
import copy

import utils.dbpedia_interface as db_interface
import utils.natural_language_utilities as nlutils

#Prerequisites
dbp = None	#DBpedia interface object
relevent_properties = open('resources/relation_whitelist.txt').read().split('\n')
sparqls = []
# relevent_entities = open('important_entities.txt','r').read().split('\n')


#Subgraph required
# 	?ent3n ?prop1n ?ent4n . 
# 	?ent2n ?prop2n ?ent1n .
# 	?ent1n ?prop1n <%(?ent0)s> . 
# 	<%(?ent0)s> ?prop1 ?ent1 .
# 	?ent1 ?prop2 ?ent2 .
# 	?ent3 ?prop3 ?ent2 .
# 	?ent4 ?prop1 ?ent5 .


#Some relevent SPARQL Queries
one_triple_right = '''
			SELECT DISTINCT ?prop1 ?ent1 
			WHERE { 
				<%(ent0)s> ?prop1 ?ent1 
			}'''
one_triple_left = '''
			SELECT DISTINCT ?prop1n ?ent1n
			WHERE {
				?ent1n ?prop1n <%(ent0)s>
			}'''
second_triple_right = '''
			SELECT DISTINCT ?prop2 ?ent2
			WHERE {
				<%(ent0)s> <%(prop1)s> <%(ent1)s> . 
				<%(ent1)s> ?prop2 ?ent2
			}'''
second_triple_left = '''
			SELECT DISTINCT ?prop2n ?ent2n
			WHERE {
				<%(ent1n)s> <%(prop1n)s> <%(ent0)s> . 
				?ent2n ?prop2n <%(ent1n)s>
			}'''



def get_local_subgraph(uri):
	'''
		The function used to query the subgraph of an entity. Once this information is collected and stored in the "subgraph" dictionary, it can be used to fill templates.
		TODO: take care of literals
	'''
	#Declaring some variables
	global dbp
	subgraph = {
		'ent1':[],
		'prop1':[],
		'ent1n':[],
		'prop1n':[],
		'ent2':{},
		'prop2':{},
		'ent2n': {},
		'prop2n': {}
	}

	#Get the connected entities and properties: ent0 prop1 ent1
	results = dbp.shoot_custom_query(one_triple_right % {'ent0': uri})
	
	# Parse and get the ent1, prop1
	for result in results[u'results'][u'bindings']:
		# If the property is an important property, fetch the entities and the properties
		if result[u'prop1'][u'value'].split('/')[-1] in relevent_properties:
			subgraph['ent1'].append(result[u'ent1'][u'value'])
			subgraph['prop1'].append(result[u'prop1'][u'value'])

	#Get the connected entities and properties: ent1n prop1n ent0
	results = dbp.shoot_custom_query(one_triple_left % {'ent0': uri})

	#Parse
	for result in results[u'results'][u'bindings']:
		# If the property is an important property, fetch the entities and the properties
		if result[u'prop1n'][u'value'].split('/')[-1] in relevent_properties:
			subgraph['ent1n'].append(result[u'ent1n'][u'value'])
			subgraph['prop1n'].append(result[u'prop1n'][u'value'])


	#For every ent1, prop1 pair, now collect ent2, prop2 pairs
	for i in range(len(subgraph['ent1'])):
		#Only for the readibility
		ent1 = subgraph['ent1'][i]
		prop1 = subgraph['prop1'][i]

		#Ensure that this isn't being done for literals
		if not nlutils.has_url(ent1) and not nlutils.has_shorthand(ent1):
			continue

		#Declaring the lists for every entity
		subgraph['ent2'][ent1] = []
		subgraph['prop2'][ent1] = []

		#Shoot n Parse
		results = dbp.shoot_custom_query(second_triple_right % {'ent0': uri, 'prop1': prop1, 'ent1': ent1})
		for result in results[u'results'][u'bindings']:
			if result[u'prop2'][u'value'].split('/')[-1] in relevent_properties:
				subgraph['prop2'][ent1].append(result[u'prop2'][u'value'])
				subgraph['ent2'][ent1].append(result[u'ent2'][u'value'])


	#For every ent1n and prop1n, fetch ent2n and prop2n
	for i in range(len(subgraph['ent1n'])):
		ent1n = subgraph['ent1n'][i]
		prop1n = subgraph['prop1n'][i]

		#Ensure that this isn't being done for literals
		if not nlutils.has_url(ent1n) and not nlutils.has_shorthand(ent1n):
			continue

		#Declaring lists for the prop2n and ent2n
		subgraph['ent2n'][ent1n] = []
		subgraph['prop2n'][ent1n] = []

		#Shoot n parse
		results = dbp.shoot_custom_query(second_triple_left % {'ent0': uri, 'prop1n': prop1n, 'ent1n': ent1n})
		for result in results[u'results'][u'bindings']:
			if result[u'prop2n'][u'value'].split('/')[-1] in relevent_properties:
				subgraph[u'prop2n'][ent1n].append(result[u'prop2n'][u'value'])
				subgraph[u'ent2n'][ent1n].append(result[u'ent2n'][u'value'])

	return subgraph

def fill_specific_template(_template_id, _mapping):
	'''
		Using this function to map a string dict to a template.
		 -> create copy of template 
		 -> if literals, do something
		 -> push it in the list
		 -> periodic writes to disk (of the list)
	'''
	global sparql

	#TODO: Think if you wanna handle literals right here

	#Create copy of the template
	template = [x for x in templates if x['id'] == _template_id][0]
	template = copy.copy(template)
	template['query'] = template['template'] % mapping
	template['mapping'] = mapping

	print template
	raw_input()
	#Push it onto the SPARQL List
	#TODO: Add periodical writes
	sparql.append(template)

	return 0




def fill_templates(subgraph):
	templates = json.load(open('templates.py'))
	
	#TEMPLATE 1

	#For the first template, use ent1n and prop1n (their sequences are synced) to do this shizz
	for i in range(len(subgraph['ent1n'])):

		#Making the variables explicit (for the sake of readability)
		ent1n = subgraph['ent1n'][i]
		prop1n = subgraph['prop1n'][i] 

		#Create a mapping (in keeping with the templates' placeholder names)
		mapping = {'rel_to_a':prop1n,'ent_a':ent1n}

		#Throw it to a function who will put it in the list with appropriate bookkeeping
		fill_specific_template(_template_id = 1, _mapping = _mapping)

	# #TEMPLATE 2
	# for i in range
	


				





In [2]:



dbp =  db_interface.DBPedia(_verbose = True)
uri = 'http://dbpedia.org/resource/New_Delhi'
subgraph = get_local_subgraph(uri)

In [8]:

pprint(subgraph['ent1n'])

[u'http://dbpedia.org/resource/Prachi_Tehlan',
 u'http://dbpedia.org/resource/DD_Gyan_Darshan_1',
 u'http://dbpedia.org/resource/DD_Metro',
 u'http://dbpedia.org/resource/Periyar_Maniammai_University',
 u'http://dbpedia.org/resource/Integrated_Management_College',
 u'http://dbpedia.org/resource/Bharati_Vidyapeeth_Institute_Of_Management_and_Research',
 u'http://dbpedia.org/resource/International_Management_Institute,_New_Delhi',
 u'http://dbpedia.org/resource/JD_Institute_of_Fashion_Technology',
 u'http://dbpedia.org/resource/Indian_Institute_of_Tourism_and_Travel_Management',
 u'http://dbpedia.org/resource/Interim_Government_of_India',
 u'http://dbpedia.org/resource/India',
 u'http://dbpedia.org/resource/Department_of_East_Asian_Studies,_University_of_Delhi',
 u'http://dbpedia.org/resource/Faculty_of_Law,_University_of_Delhi',
 u'http://dbpedia.org/resource/Fore_School_of_Management',
 u'http://dbpedia.org/resource/Indian_Council_of_Agricultural_Research',
 u"http://dbpedia.org/resour